In [629]:
import re
import requests
import pandas as pd
import numpy as np
import urllib
import json
from bs4 import BeautifulSoup, NavigableString

#### Below is the wikipedia api call for a category search:

`http://en.wikipedia.org/w/api.php?action=query&format=json&list=categorymembers&cmtitle=Category%3A+machine+learning&cmlimit=max`

`action=query`: query the wikipedia api

`format=json`: return a json format

`list=categorymembers`: List of pages that belong to a given category, ordered by page sort title

`cmtitle=Category%3A+machine+learning`: title of category

`climit=max`: return up to the maximum amount of responses (500)

You may use this to get page titles from the wikipedia API. Things to watch out for:
* The responses contain categories
* You will want to fetch articles in those subcategories

The API's detailed documentation can be found [here](https://www.mediawiki.org/wiki/API:Main_page)

#### Make a function that formats a request for pages of a category

In [630]:
def generate_category(category):
    category = re.sub('\s', '_', category)
    return category


In [631]:
category = 'Category:Machine learning'

In [632]:
bs_cat = 'Category:Business software'

In [633]:
ml = generate_category(category)

In [634]:
ml

'Category:Machine_learning'

In [635]:
bs = generate_category(bs_cat)

In [636]:
bs

'Category:Business_software'

In [637]:
def cat_pages_depth(cat_name, max_depth=3):
    
    params = {'action':'query',
          'format':'json',
          'list':'categorymembers',
          'cmtitle': generate_category(cat_name),
          'cmlimit':'max',}
    
    response = requests.get('http://en.wikipedia.org/w/api.php?', params=params)
    data = response.json()
    
    members = data['query']['categorymembers']

    pages = list(filter(lambda x: x['ns'] == 0, members))
    subpages = list(filter(lambda x: x['ns'] == 14, members))
    
    while max_depth > 0:
    
        if not subpages:    
            return pages
        else:
            for subpage in subpages:
                max_depth -=1
                pages += cat_pages_depth(subpage['title'], max_depth)
            
    return pages

In [667]:
dummy = pd.DataFrame(cat_pages_depth(ml))

In [823]:
dummy_2 = pd.DataFrame(cat_pages_depth(bs))

In [642]:
dummy.shape

(1075, 3)

In [670]:
dummy['category'] = 'Machine learning'

In [671]:
dummy.head()

,ns,pageid,title,category
0,0,43385931,Data exploration,Machine learning
1,0,49082762,List of datasets for machine learning research,Machine learning
2,0,233488,Machine learning,Machine learning
3,0,53587467,Outline of machine learning,Machine learning
4,0,3771060,Accuracy paradox,Machine learning


In [824]:
dummy_2['category'] = 'Business software'

In [825]:
dummy_2.head()

,ns,pageid,title,category
0,0,1037763,Business software,Business software
1,0,41270069,AccuSystems,Business software
2,0,5211212,Active policy management,Business software
3,0,28502793,Alexandria (library software),Business software
4,0,44133735,Alteryx,Business software


In [674]:
def cleaner(message):
    message = re.sub('\.+', ' ', message)
    message = re.sub('[^a-z0-9 ]','', message.lower())
    message = re.sub('\d+','NUMBER ',message)
    message = re.sub('\s+',' ',message)
    return message

In [677]:
text_list = []
for _,row in dummy.iterrows():
    page = row['pageid']
    query = 'https://en.wikipedia.org/w/api.php?format=json&\
    action=query&prop=extracts&explaintext&pageids={}'.format(page)
    response = requests.get(query)
    page_info = response.json()
    page_text = page_info['query']['pages'][str(page)]['extract']
    page_text = cleaner(page_text)
    text_list.append(page_text)
    

In [827]:
bs_text_list = []
for _,row in dummy_2.iterrows():
    page = row['pageid']
    query = 'https://en.wikipedia.org/w/api.php?format=json&\
    action=query&prop=extracts&explaintext&pageids={}'.format(page)
    response = requests.get(query)
    page_info = response.json()
    page_text = page_info['query']['pages'][str(page)]['extract']
    page_text = cleaner(page_text)
    bs_text_list.append(page_text)

In [678]:
dummy['text'] = text_list

In [828]:
dummy_2['text'] = bs_text_list

In [681]:
dummy.head(3)

ns    pageid                                           title  \
0  0   43385931  Data exploration                                 
1  0   49082762  List of datasets for machine learning research   
2  0   233488    Machine learning                                 

           category  \
0  Machine learning   
1  Machine learning   
2  Machine learning   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         

In [680]:
dummy.shape

(1075, 5)

In [830]:
dummy_2.head(3)

ns    pageid                     title           category  \
0  0   1037763   Business software         Business software   
1  0   41270069  AccuSystems               Business software   
2  0   5211212   Active policy management  Business software   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  

In [831]:
dummy_2.shape

(1918, 5)

In [832]:
ml_df = dummy.copy()

In [833]:
bs_df = dummy_2.copy()

In [644]:
# def cleaner(whatever):
#     """
#     used in page_hunter function to clean urls
#     """
#     whatever = re.sub('\s', '_', whatever)
#     whatever = re.sub('\+', '%2B', whatever)
#     whatever = re.sub('\&', '%26', whatever)
#     return whatever

In [645]:
# def page_hunter(page):
#     url = """
#     http://en.wikipedia.org/w/api.php?action=query&\
#               format=json&prop=extracts&titles={}&rvprop=content""".format(cleaner(page))
#     url = re.sub('\s', '', url)
#     r = requests.get(url)
#     key = list(r.json()['query']['pages'].keys())[0]
#     return r.json()['query']['pages'][key]['extract']
    

In [646]:
# def get_pages(scraped_df):
#     """
#     maps titles to newly created text column in df
#     """
    
#     scraped_df['text'] = scraped_df['title'].map(lambda x: page_hunter(x))
    
#     return scraped_df
    
    
# #     for i, page in enumerate(scraped_df.title):
# #         page_to_df(page)
    
    

In [647]:
# ml_df_all = get_pages(ml_df)

In [692]:
ml_df_no_ns = dummy.drop(['ns'], axis=1)

In [834]:
bs_df_no_ns = dummy_2.drop(['ns'], axis=1)

In [693]:
ml_df_no_ns.head(4)

pageid                                           title          category  \
0  43385931  Data exploration                                Machine learning   
1  49082762  List of datasets for machine learning research  Machine learning   
2  233488    Machine learning                                Machine learning   
3  53587467  Outline of machine learning                     Machine learning   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               

In [694]:
ml_json = ml_df_no_ns.to_json()

In [835]:
bs_json = bs_df_no_ns.to_json()

In [726]:
# for key in ml_dict.keys():
#     if type(key) is not str:
#         try:
#             ml_dict[str(key)] = ml_dict[key]
#         except:
#             try:
#                 ml_dict[repr(key)] = ml_dict[key]
#             except:
#                 pass
#         del ml_dict[key]

In [778]:
ml_dict.keys()

dict_keys(['pageid', 'title', 'category', 'text'])

In [794]:

from collections import OrderedDict
data = json.loads(ml_json, object_pairs_hook=OrderedDict)

In [836]:
data_2 = json.loads(bs_json, object_pairs_hook=OrderedDict)

In [796]:
data.keys()

odict_keys(['pageid', 'title', 'category', 'text'])

In [837]:
data_2.keys()

odict_keys(['pageid', 'title', 'category', 'text'])

### MONGO

In [1]:
!conda install --yes --quiet pymongo



Package plan for installation in environment /opt/conda:

The following NEW packages will be INSTALLED:

    libgfortran-ng: 7.2.0-h9f7466a_2  defaults   
    libopenblas:    0.2.20-hae245c1_3 defaults   
    pymongo:        3.4.0-py36_0      defaults   

The following packages will be DOWNGRADED:

    numexpr:        2.6.4-py36_0      conda-forge --> 2.6.2-py36_nomklhd302951_1 defaults [nomkl]



In [2]:
import pymongo

In [754]:
client = pymongo.MongoClient('35.167.189.162', 27016)

### Databases on Our MongoDB Server

In [786]:
coll_ref.drop()

In [787]:
client.database_names()

['admin', 'local', 'test', 'twitter']

In [788]:
test_db = client.test


### Make a new Database called `project4_database`


In [789]:
db_ref = client.project4_database

#### Create a reference to `my_project4_collection`

In [790]:
coll_ref = db_ref.my_project4_collection

In [839]:
coll_ref_bs = db_ref.my_project4_collection_bs

#### Show `databases` and the `collections`

In [840]:
client.database_names(), db_ref.collection_names()

(['admin', 'local', 'project4_database', 'test', 'twitter'],
 ['my_project4_collection'])

In [797]:

mongo_ml = coll_ref.insert_one(data)


In [841]:
mongo_bs = coll_ref_bs.insert_one(data_2)

In [817]:
cursor = coll_ref.find_one()

cursor

sample_docs = (cursor)

In [842]:
cursor_2 = coll_ref_bs.find_one()

cursor_2

sample_docs_2 = (cursor_2)

In [818]:
type(sample_docs)

dict

In [819]:
sample_docs.keys()

dict_keys(['_id', 'pageid', 'title', 'category', 'text'])

In [843]:
sample_docs_2.keys()

dict_keys(['_id', 'pageid', 'title', 'category', 'text'])

In [820]:
from_mongo_ml_df = pd.DataFrame(sample_docs)

In [844]:
from_mongo_bs_df = pd.DataFrame(sample_docs_2)

In [822]:
from_mongo_ml_df.shape

(1075, 5)

In [845]:
from_mongo_bs_df.shape

(1918, 5)

In [846]:
from_mongo_bs_df.head()

_id           category    pageid  \
0     5a165d59608b9000b627015d  Business software  1037763    
1     5a165d59608b9000b627015d  Business software  41270069   
10    5a165d59608b9000b627015d  Business software  35959361   
100   5a165d59608b9000b627015d  Business software  11028436   
1000  5a165d59608b9000b627015d  Business software  25822348   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                